# DINOv3 bbox sanity test

In [ ]:
# Setup (Colab): installs if missing
try:
    import torch, transformers, timm, pillow
except Exception:
    !pip -q install -U torch torchvision torchaudio transformers timm pillow matplotlib numpy


In [ ]:
# Paths + params
IMAGE_PATHS = [
    # e.g. "/content/drive/MyDrive/Dazzl/test_inputs/model.jpg",
]
GARMENT_HINT = None   # e.g. "joggers" or leave None to derive from path
DINO_MODEL_ID = "facebook/dinov3-vits16-pretrain-lvd1689m"
DINO_PERCENTILE = 80
CROP_PADDING = 40


In [ ]:
import os, math
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

from dino import (
    detect_garment_region_with_dinov3,
    heatmap_to_pil,
    draw_bbox,
    infer_garment_type_from_path,
)

WHITE_RGB = (255, 255, 255)

def make_square_crop_with_padding(full_img: Image.Image, bbox, fill=WHITE_RGB):
    x0, y0, x1, y1 = map(int, bbox)
    bw, bh = x1 - x0, y1 - y0
    side = max(bw, bh, 1)
    extra_w = side - bw
    extra_h = side - bh
    sq_x0 = x0 - extra_w // 2
    sq_y0 = y0 - extra_h // 2
    sq_x1 = sq_x0 + side
    sq_y1 = sq_y0 + side

    img_w, img_h = full_img.size
    src_x0 = max(0, sq_x0)
    src_y0 = max(0, sq_y0)
    src_x1 = min(img_w, sq_x1)
    src_y1 = min(img_h, sq_y1)

    crop = full_img.crop((src_x0, src_y0, src_x1, src_y1))
    canvas = Image.new("RGB", (side, side), fill)
    paste_x = src_x0 - sq_x0
    paste_y = src_y0 - sq_y0
    canvas.paste(crop, (paste_x, paste_y))
    return canvas, [sq_x0, sq_y0, sq_x1, sq_y1]


In [ ]:
def run_one(path, garment_hint=None):
    img = Image.open(path).convert("RGB")
    gh = garment_hint or infer_garment_type_from_path(path)
    bbox, heat = detect_garment_region_with_dinov3(
        img,
        garment_hint=gh,
        score_percentile=DINO_PERCENTILE,
        padding=CROP_PADDING,
    )
    vis = draw_bbox(img, bbox, color=(255, 99, 71), width=10)
    hm = heatmap_to_pil(heat)
    square_crop, square_box = make_square_crop_with_padding(img, bbox, fill=WHITE_RGB)

    print(f"Garment hint: {gh}")
    print(f"Bbox: {bbox}  | Square: {square_box}  | Image: {img.size}")

    plt.figure(figsize=(12, 8))
    plt.subplot(2,2,1); plt.imshow(vis); plt.title("DINO bbox"); plt.axis('off')
    plt.subplot(2,2,2); plt.imshow(hm); plt.title("DINO heatmap"); plt.axis('off')
    plt.subplot(2,2,3); plt.imshow(square_crop); plt.title("Square crop (padded)"); plt.axis('off')
    plt.subplot(2,2,4); plt.imshow(img); plt.title("Original"); plt.axis('off')
    plt.show()

for p in IMAGE_PATHS:
    if os.path.exists(p):
        run_one(p, garment_hint=GARMENT_HINT)
    else:
        print(f"⚠️ Missing: {p}")
